In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, Input
from keras.optimizers import SGD, Adam
from glob import glob

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
storm_files = sorted(glob("/home/ubuntu/track_data_ncarstorm_3km_nc_small/*.nc"))
run_times = []
valid_times = []
in_vars = ["REFL_COM_prev",
           "PSFC_prev", "T2_prev", "TD2_prev", "U10_prev", "V10_prev"]
out_vars = ["RVORT1_MAX_curr", "masks"]
in_data = []
out_data = []
for storm_file in storm_files:
    ds = xr.open_dataset(storm_file)
    in_data.append(np.stack([ds[v].values for v in in_vars], axis=-1))
    out_data.append(np.stack([ds[v].values for v in out_vars], axis=-1))
    valid_times.append(ds["time"].values)
    run_time = pd.Timestamp(storm_file.split("/")[-1].split("_")[1])
    run_times.append([run_time] * in_data[-1].shape[0])
    print(run_time)
    ds.close()
#combined_ds = xr.concat(all_ds)

2010-10-24 00:00:00
2010-10-25 00:00:00
2010-10-26 00:00:00
2010-10-27 00:00:00
2010-11-16 00:00:00
2010-11-22 00:00:00
2010-11-29 00:00:00
2010-11-30 00:00:00
2010-12-31 00:00:00
2011-01-25 00:00:00
2011-02-01 00:00:00
2011-02-24 00:00:00
2011-02-27 00:00:00
2011-02-28 00:00:00
2011-03-05 00:00:00
2011-03-08 00:00:00
2011-03-09 00:00:00
2011-03-10 00:00:00
2011-03-22 00:00:00
2011-03-23 00:00:00
2011-03-26 00:00:00
2011-03-27 00:00:00
2011-03-29 00:00:00
2011-03-31 00:00:00
2011-04-03 00:00:00
2011-04-04 00:00:00
2011-04-08 00:00:00
2011-04-09 00:00:00
2011-04-10 00:00:00
2011-04-11 00:00:00
2011-04-14 00:00:00
2011-04-15 00:00:00
2011-04-16 00:00:00
2011-04-18 00:00:00
2011-04-19 00:00:00
2011-04-20 00:00:00
2011-04-21 00:00:00
2011-04-22 00:00:00
2011-04-23 00:00:00
2011-04-24 00:00:00
2011-04-25 00:00:00
2011-04-26 00:00:00
2011-04-27 00:00:00
2011-04-28 00:00:00
2011-04-30 00:00:00
2011-05-09 00:00:00
2011-05-10 00:00:00
2011-05-11 00:00:00
2011-05-12 00:00:00
2011-05-21 00:00:00


2016-04-26 00:00:00
2016-04-27 00:00:00
2016-04-28 00:00:00
2016-04-29 00:00:00
2016-05-01 00:00:00
2016-05-02 00:00:00
2016-05-03 00:00:00
2016-05-04 00:00:00
2016-05-05 00:00:00
2016-05-07 00:00:00
2016-05-08 00:00:00
2016-05-09 00:00:00
2016-05-10 00:00:00
2016-05-11 00:00:00
2016-05-12 00:00:00
2016-05-16 00:00:00
2016-05-17 00:00:00
2016-05-21 00:00:00
2016-05-22 00:00:00
2016-05-24 00:00:00
2016-05-25 00:00:00
2016-05-26 00:00:00
2016-05-27 00:00:00
2016-05-28 00:00:00
2016-05-29 00:00:00
2016-05-31 00:00:00
2016-06-02 00:00:00
2016-06-05 00:00:00
2016-06-08 00:00:00
2016-06-10 00:00:00
2016-06-13 00:00:00
2016-06-14 00:00:00
2016-06-15 00:00:00
2016-06-16 00:00:00
2016-06-17 00:00:00
2016-06-19 00:00:00
2016-06-20 00:00:00
2016-06-21 00:00:00
2016-06-22 00:00:00
2016-06-23 00:00:00
2016-06-24 00:00:00
2016-06-25 00:00:00
2016-06-26 00:00:00
2016-06-28 00:00:00
2016-06-29 00:00:00
2016-07-04 00:00:00
2016-07-05 00:00:00
2016-07-06 00:00:00
2016-07-07 00:00:00
2016-07-08 00:00:00


In [ ]:
ds = xr.open_dataset(storm_files[0])
ds["time"].values

In [ ]:
np.stack([ds[v].values for v in in_vars], axis=-1).shape

In [3]:
all_in_data = np.vstack(in_data)

In [4]:
all_in_data.shape

(173675, 32, 32, 6)